In [2]:
import numpy as np
from keras.preprocessing.text import Tokenizer
import glob
import shutil

Using TensorFlow backend.


Split files of a specific categories. This function add new file texts to the existing train/val/test text sets.

In [144]:
def split_files(train_size, val_size, test_size,categories) :
    #ex categories = ['00','01']
    for cat in categories : 
        cnt_cat = 0
        for file in glob.glob('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/data_plain_and_tokenized/'+cat+'*[!_tokenized].txt') :
            cnt_cat+=1
            if cnt_cat <= train_size : 
                destination = '/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/train_list'
                shutil.copy2(file, destination)
            if train_size < cnt_cat and cnt_cat<= train_size+val_size  : 
                destination = '/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/val_list'
                shutil.copy2(file, destination)      
            if train_size+val_size < cnt_cat and cnt_cat<= train_size+val_size+test_size : 
                destination = '/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/test_list'
                shutil.copy2(file, destination)     
    

In [146]:
split_files(1110,185,185,categories = ['02'])

In [42]:
from keras.preprocessing.text import Tokenizer
import os

Create document term matrices from the set of texts. It includes a tokenizer module.

In [112]:
def create_inputs(max_n_words, mode_='tfidf') : 
    
    text_train = []
    target_train = []
    text_val = []
    target_val = []
    text_test = []
    target_test = []   
    

    for link in glob.glob('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/train_list/*.txt') : 
        file = open(link,'r')
        text_train.append(file.read())
        target_train.append(int(os.path.basename(file.name)[0:2]))

        
    for link in glob.glob('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/val_list/*.txt') : 
        file = open(link,'r')
        text_val.append(file.read())
        target_val.append(int(os.path.basename(file.name)[0:2]))   

    for link in glob.glob('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/test_list/*.txt') : 
        file = open(link,'r')
        text_test.append(file.read())
        target_test.append(int(os.path.basename(file.name)[0:2]))   
        
    
    tokenizer = Tokenizer(num_words = max_n_words, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
    tokenizer.fit_on_texts(text_train)
    
 
    x_train = tokenizer.texts_to_matrix(text_train, mode=mode_)
    y_train = np.asarray(target_train)
  
    x_val = tokenizer.texts_to_matrix(text_val, mode=mode_)
    y_val = np.asarray(target_val)
    
    x_test = tokenizer.texts_to_matrix(text_test, mode=mode_)
    y_test = np.asarray(target_test)

    return x_train, y_train, x_val, y_val, x_test, y_test

    

In [98]:
import pickle

Save the list of train/val/test matrices with pickle.
I saved different .dat files adding gradually data from new topics.

In [147]:
def dump_inputs(num_set,max_n_words=20000,mode_='tfidf') : 
    x_train, y_train, x_val, y_val, x_test, y_test = create_inputs(max_n_words, mode_)
    pickle.dump([x_train, y_train, x_val, y_val, x_test, y_test], \
        open('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/dumps/'+str(num_set)+'.dat','wb'))

In [148]:
dump_inputs(6,max_n_words=20000,mode_='tfidf') 

In [123]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.calibration import CalibratedClassifierCV
import numpy as np
from matplotlib import pyplot

In [124]:
def run_svm(num_set): 
    print('Build SVM model ...')
    [x_train, y_train, x_val, y_val, x_test, y_test] = pickle.load(open('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/dumps/'+str(num_set)+'.dat', "rb"))
    svm = LinearSVC(C=0.0001)
    svm = CalibratedClassifierCV(svm)
    svm.fit(x_train, y_train)
    y_pred = svm.predict(x_train)
    acc = accuracy_score(y_train, y_pred)
    y_pred = svm.predict(x_val)
    val_acc = accuracy_score(y_val, y_pred)
    
    #print('ypred  :', y_pred)
    #print('yval :', y_val)
    print('acc :', acc)
    print('val acc :', val_acc)
    print(classification_report(y_val, y_pred))
    print(confusion_matrix(y_val, y_pred))
    return svm.predict_proba(np.concatenate((x_train, x_val), axis=0))[:, 1], \
           svm.predict_proba(x_test)[:, 1], \
           np.concatenate((y_train, y_val), axis=0)

In [149]:
run_svm(6)

Build SVM model ...
acc : 0.9956947162426615
val acc : 0.9563451776649746
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       400
           1       0.96      0.96      0.96       400
           2       0.91      0.94      0.93       185

    accuracy                           0.96       985
   macro avg       0.95      0.95      0.95       985
weighted avg       0.96      0.96      0.96       985

[[383   9   8]
 [  6 385   9]
 [  2   9 174]]


(array([9.88614895e-01, 9.50049373e-03, 1.08883149e-08, ...,
        9.67898775e-01, 9.99996239e-01, 9.99593072e-01]),
 array([1.22210290e-01, 7.87530963e-03, 2.60992101e-03, 2.93085164e-02,
        7.33271805e-03, 4.62672297e-02, 4.19226418e-04, 2.68593415e-05,
        6.94316448e-02, 6.66150267e-02, 1.63035649e-04, 5.35729089e-03,
        6.25912665e-02, 5.67519398e-02, 5.82779835e-02, 2.14196974e-03,
        1.09182104e-03, 1.50678764e-04, 3.68603511e-02, 3.28735623e-02,
        6.86004424e-04, 8.80672471e-07, 3.86202938e-02, 1.58866816e-02,
        9.77616896e-01, 2.67139319e-01, 2.35642478e-02, 3.49731946e-02,
        2.69801931e-03, 9.13011580e-07, 1.77712669e-03, 1.95571370e-01,
        6.04440043e-02, 2.89996652e-01, 5.96694357e-07, 1.28307874e-07,
        3.72434180e-06, 3.80285518e-02, 2.12960221e-04, 1.09617155e-05,
        8.32738109e-02, 1.20388560e-01, 3.37199006e-09, 7.23496620e-02,
        1.93209601e-07, 1.57673736e-03, 7.32547390e-10, 4.44394027e-03,
        3.2549066

In [115]:
from sklearn.ensemble import RandomForestClassifier

In [151]:
def run_RandomForest(num_set):
    print('Build Random Forest model ...')
    [x_train, y_train, x_val, y_val, x_test, y_test] = pickle.load(open('/Users/andrealequin/Desktop/Stage_NLP/Simple_extraction/dumps/'+str(num_set)+'.dat', "rb"))
    clf = RandomForestClassifier(max_depth=3, random_state=0)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_train)
    acc = accuracy_score(y_train, y_pred)
    y_pred = clf.predict(x_val)
    val_acc = accuracy_score(y_val, y_pred)
    
    #print('ypred  :', y_pred)
    #print('yval :', y_val)
    print('acc :', acc)
    print('val acc :', val_acc)
    print(classification_report(y_val, y_pred))
    print(confusion_matrix(y_val, y_pred))
    
    return clf.predict_proba(np.concatenate((x_train, x_val), axis=0))[:, 1], \
           clf.predict_proba(x_test)[:, 1], \
           np.concatenate((y_train, y_val), axis=0)    
    

In [152]:
run_RandomForest(6)

Build Random Forest model ...
acc : 0.8544031311154598
val acc : 0.8598984771573605
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       400
           1       0.77      0.98      0.86       400
           2       0.99      0.45      0.62       185

    accuracy                           0.86       985
   macro avg       0.90      0.79      0.81       985
weighted avg       0.88      0.86      0.85       985

[[370  30   0]
 [  5 394   1]
 [ 14  88  83]]


(array([0.50606787, 0.37070115, 0.16363446, ..., 0.5256839 , 0.67359348,
        0.61410548]),
 array([0.353432  , 0.39005624, 0.26850971, 0.30427064, 0.29162359,
        0.38357769, 0.29403834, 0.1934012 , 0.33402203, 0.34108788,
        0.17637729, 0.32103671, 0.36157383, 0.36398614, 0.35571809,
        0.22805504, 0.26842215, 0.20438969, 0.31751791, 0.28180906,
        0.29944466, 0.16245234, 0.31223305, 0.26677033, 0.42785633,
        0.45668858, 0.33513221, 0.33034912, 0.31160747, 0.16463907,
        0.29842324, 0.36635591, 0.34518454, 0.41069117, 0.18917008,
        0.19809496, 0.20047549, 0.43001964, 0.34211357, 0.2625248 ,
        0.36773052, 0.3806058 , 0.09939254, 0.35318539, 0.1814722 ,
        0.2837676 , 0.15224288, 0.2748365 , 0.11341747, 0.36509165,
        0.3904192 , 0.23398676, 0.13214882, 0.23601742, 0.3213297 ,
        0.32154667, 0.32724749, 0.25255004, 0.31822743, 0.34498081,
        0.34358187, 0.26122704, 0.26469349, 0.39812827, 0.21579451,
        0.28498135, 0